In [8]:
import zipfile
import os
import cv2
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import pickle

Function to extract the zip file

In [22]:
def extract_zip(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Extracted zip file to {extract_to}")

Function to load images from extracted folder and assign labels based on subfolder names


In [23]:
def load_images_from_folder(main_folder, img_size=(32, 32)):
    images = []
    labels = []
    classes = os.listdir(main_folder)  # Should contain folders named '1' and '0'

    for class_name in classes:
        class_folder = os.path.join(main_folder, class_name)
        if os.path.isdir(class_folder):  # Ensure it's a folder
            for filename in os.listdir(class_folder):
                img_path = os.path.join(class_folder, filename)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale
                if img is not None:
                    img_resized = cv2.resize(img, img_size)  # Resize image to the specified size
                    img_flattened = img_resized.flatten()    # Flatten image to 1D array
                    images.append(img_flattened)
                    labels.append(int(class_name))  # Convert folder name '1' or '0' to integer label

    return np.array(images), np.array(labels)  # Return images and labels as NumPy arrays


Path to the zip file

In [24]:
zip_path = 'archive.zip'
extract_to = 'dataset'  # Path to extract the zip contents

Extract the zip file

In [32]:
extract_zip(zip_path, extract_to)

Extracted zip file to dataset


Path to the extracted folder

In [33]:
extracted_folder_path = os.path.join(extract_to, 'myData')

training data

In [35]:
X, y = load_images_from_folder(extracted_folder_path)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training data shape: {X_train.shape}, Testing data shape: {X_test.shape}")

Training data shape: (520, 1024), Testing data shape: (131, 1024)


In [36]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [37]:
model = MLPClassifier(hidden_layer_sizes=(64, 64), max_iter=250, verbose=True)
model.fit(X_train, y_train)

Iteration 1, loss = 0.75050575
Iteration 2, loss = 0.47028530
Iteration 3, loss = 0.39870355
Iteration 4, loss = 0.35169739
Iteration 5, loss = 0.30966274
Iteration 6, loss = 0.27694298
Iteration 7, loss = 0.24724036
Iteration 8, loss = 0.22320811
Iteration 9, loss = 0.19604205
Iteration 10, loss = 0.18357024
Iteration 11, loss = 0.16013820
Iteration 12, loss = 0.14368167
Iteration 13, loss = 0.12804846
Iteration 14, loss = 0.11564705
Iteration 15, loss = 0.10072212
Iteration 16, loss = 0.09025073
Iteration 17, loss = 0.07984967
Iteration 18, loss = 0.06936317
Iteration 19, loss = 0.06172082
Iteration 20, loss = 0.05419918
Iteration 21, loss = 0.04776209
Iteration 22, loss = 0.04160913
Iteration 23, loss = 0.03759251
Iteration 24, loss = 0.03272135
Iteration 25, loss = 0.02956240
Iteration 26, loss = 0.02611343
Iteration 27, loss = 0.02330541
Iteration 28, loss = 0.02094284
Iteration 29, loss = 0.01880007
Iteration 30, loss = 0.01721938
Iteration 31, loss = 0.01554147
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(64, 64), max_iter=250, verbose=True)

predictions

In [38]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

Evaluate accuracy

In [39]:
print('Training accuracy:', accuracy_score(y_train, y_train_pred) * 100, '%')
print('Testing accuracy:', accuracy_score(y_test, y_test_pred) * 100, '%')

Training accuracy: 100.0 %
Testing accuracy: 77.86259541984732 %


Example prediction

In [40]:
print('Prediction for one sample:', model.predict([X_test[55]]), y_test[55])

Prediction for one sample: [0] 0


save

In [41]:
model_filename = 'trained_model.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(model, file)  # Save the model
print(f"Model saved as {model_filename}")

Model saved as trained_model.pkl


load model

In [42]:
with open(model_filename, 'rb') as file:
    loaded_model = pickle.load(file)

Make predictions with the loaded model

In [43]:
y_test_pred_loaded_model = loaded_model.predict(X_test)

# Evaluate the accuracy of the loaded model
print('Testing accuracy with loaded model:', accuracy_score(y_test, y_test_pred_loaded_model) * 100, '%')

Testing accuracy with loaded model: 77.86259541984732 %
